In [3]:
import coiled

In [4]:
cluster = coiled.Cluster(name="quickstart-example", n_workers=10)

/Users/powers/.pyenv/versions/3.7.5/lib/python3.7/site-packages/rich/live.py:227: UserWarning: install "ipywidgets" for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')


Found software environment build


/Users/powers/.pyenv/versions/3.7.5/lib/python3.7/site-packages/coiled/cluster.py:629: UserWarning: Unable to locate credentials
  warnings.warn(str(e))


In [5]:
from dask.distributed import Client

client = Client(cluster)

/Users/powers/.pyenv/versions/3.7.5/lib/python3.7/site-packages/distributed/client.py:1148: VersionMismatchWarning: Mismatched versions found

+---------+-----------------------------+----------------+----------------+
| Package | client                      | scheduler      | workers        |
+---------+-----------------------------+----------------+----------------+
| blosc   | None                        | 1.10.2         | 1.10.2         |
| dask    | 2021.05.0+7.g7483d445.dirty | 2021.05.0      | 2021.05.0      |
| lz4     | None                        | 3.1.3          | 3.1.3          |
| numpy   | 1.20.3                      | 1.20.2         | 1.20.2         |
| python  | 3.7.5.final.0               | 3.7.10.final.0 | 3.7.10.final.0 |
+---------+-----------------------------+----------------+----------------+
  warnings.warn(version_module.VersionMismatchWarning(msg[0]["warning"]))


In [6]:
dtype = {
    "payment_type": "UInt8",
    "VendorID": "UInt8",
    "passenger_count": "UInt8",
    "RatecodeID": "UInt8",
    "store_and_fwd_flag": "category",
    "PULocationID": "UInt16",
    "DOLocationID": "UInt16",
}

In [7]:
import dask.dataframe as dd

df = dd.read_csv(
    "s3://nyc-tlc/trip data/yellow_tripdata_2019-01.csv",
    parse_dates=["tpep_pickup_datetime", "tpep_dropoff_datetime"],
    dtype=dtype,
    storage_options={"anon": True},
    blocksize="16 MiB",
).persist()

/Users/powers/.pyenv/versions/3.7.5/lib/python3.7/site-packages/pandas/compat/__init__.py:97: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


In [8]:
df.npartitions

41

In [9]:
df_unpersisted = dd.read_csv(
    "s3://nyc-tlc/trip data/yellow_tripdata_2019-01.csv",
    parse_dates=["tpep_pickup_datetime", "tpep_dropoff_datetime"],
    dtype=dtype,
    storage_options={"anon": True},
    blocksize="16 MiB",
)

In [10]:
df_no_blocksize = dd.read_csv(
    "s3://nyc-tlc/trip data/yellow_tripdata_2019-01.csv",
    parse_dates=["tpep_pickup_datetime", "tpep_dropoff_datetime"],
    dtype=dtype,
    storage_options={"anon": True},
)

In [11]:
df_no_blocksize.npartitions

11

In [12]:
dask_benchmarks = {
    'duration': [],  # in seconds
    'task': [],
}

In [13]:
import time
 
def benchmark(f, df, benchmarks, name, **kwargs):
    """Benchmark the given function against the given DataFrame.
    
    Parameters
    ----------
    f: function to benchmark
    df: data frame
    benchmarks: container for benchmark results
    name: task name
    
    Returns
    -------
    Duration (in seconds) of the given operation
    """
    start_time = time.time()
    ret = f(df, **kwargs)
    benchmarks['duration'].append(time.time() - start_time)
    benchmarks['task'].append(name)
    print(f"{name} took: {benchmarks['duration'][-1]} seconds")
    return benchmarks['duration'][-1]
 
def get_results(benchmarks):
    """Return a pandas DataFrame containing benchmark results."""
    return pd.DataFrame.from_dict(benchmarks)

In [14]:
def count(df=None):
    return len(df)

In [16]:
benchmark(count, df=df, benchmarks=dask_benchmarks, name='count_persisted')

count_persisted took: 0.2017679214477539 seconds


0.2017679214477539

In [17]:
benchmark(count, df=df_unpersisted, benchmarks=dask_benchmarks, name='count_unpersisted')

count_unpersisted took: 5.543921947479248 seconds


5.543921947479248

In [18]:
benchmark(count, df=df_no_blocksize, benchmarks=dask_benchmarks, name='count_no_blocksize')

count_no_blocksize took: 6.123486042022705 seconds


6.123486042022705

In [19]:
benchmark(count, df=df_no_blocksize.repartition(1), benchmarks=dask_benchmarks, name='count_one_partition')

count_one_partition took: 8.899395227432251 seconds


8.899395227432251

In [20]:
dask_benchmarks

{'duration': [0.13988685607910156,
  0.2017679214477539,
  5.543921947479248,
  6.123486042022705,
  8.899395227432251],
 'task': ['persisted_count',
  'count_persisted',
  'count_unpersisted',
  'count_no_blocksize',
  'count_one_partition']}

In [23]:
df.to_parquet("./data/yellow_tripdata_parquet")

ModuleNotFoundError: No module named 'fastparquet'

distributed.client - ERROR - Failed to reconnect to scheduler after 10.00 seconds, closing client
_GatheringFuture exception was never retrieved
future: <_GatheringFuture finished exception=CancelledError()>
concurrent.futures._base.CancelledError
